# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [23]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [7]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [ ]:
# to do: homework :)

## Question 1: What is the sum of the outputs of the generator for limit = 5?

In [14]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

sum = 0
for sqrt_value in generator:
    sum += sqrt_value

print('Sum of outputs for limit = 5: %.3f' % sum)


Sum of outputs for limit = 5: 8.382


## Question 2: What is the 13th number yielded

In [13]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    latest_value = sqrt_value

print('13th number yielded: %.3f' % latest_value)

13th number yielded: 3.606


## Question 3. Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

In [21]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i,
               "Name": f"Person_{i}",
               "Age": 25 + i,
               "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i,
               "Name": f"Person_{i}",
               "Age": 30 + i,
               "City": "City_B",
               "Occupation": f"Job_{i}"}

# Define connection to load data to
pipeline = dlt.pipeline(pipeline_name="people_data",
						destination='duckdb',
						dataset_name='people')

# Create or replace a table with people_1() in the duckdb database
info_1 = pipeline.run(people_1(),
                      table_name="info",
                      write_disposition="replace")

# Append people_2() to the existing table in the duckdb database
info_2 = pipeline.run(people_2(),
                      table_name="info",
                      write_disposition="append",
                      primary_key="id")

# Setup connection to database for querying
con = duckdb.connect("people_data.duckdb")

# Preview the table
con.sql("SELECT * FROM people.info").show()

# Calculate the sum of ages of all people
con.sql("SELECT SUM(age) FROM people.info").show()

# Close connection
con.close()

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707617926.1184857 │ mefFHgxjlN1zfQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707617926.1184857 │ WllkQXMGnflBWQ │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1707617926.1184857 │ FJnEnFQuYIuwyg │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1707617926.1184857 │ Y3UTdsjLae0NwA │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1707617926.1184857 │ PJWcx4nWLQaQ4w │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1707617926.882449  │ c3xb+Qu77eUHBQ │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1707617926.882449  │ k4cmiZ3D1kAjlQ │ Job_4      │
│     5 │ 

## Question 4. Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

In [22]:
# Define connection to load data to
pipeline_merge = dlt.pipeline(pipeline_name="people_data_merge",
                              destination='duckdb',
                              dataset_name='people')

# Create or replace a table with people_1() in the duckdb database
info_merge_1 = pipeline_merge.run(people_1(),
                                  table_name="info",
                                  write_disposition="replace")

# Merge people_2() to the existing table in the duckdb database
info_merge_2 = pipeline_merge.run(people_2(),
                                  table_name="info",
                                  write_disposition="merge",
                                  primary_key="id")

# Setup connection to database for querying
con = duckdb.connect("people_data_merge.duckdb")

# Preview the table
con.sql("SELECT * FROM people.info").show()
print()

# Calculate the sum of ages of all people
con.sql("SELECT SUM(age) FROM people.info").show()

# Close connection
con.close()

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1707618108.6456776 │ WnBb8Kwad4mcRA │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1707618108.6456776 │ SPhSx/2n3l3Bbw │ NULL       │
│     5 │ Person_5 │    35 │ City_B  │ 1707618109.6456246 │ JqxnzjOg76DL7Q │ Job_5      │
│     7 │ Person_7 │    37 │ City_B  │ 1707618109.6456246 │ kFbSk4h3QqeDOg │ Job_7      │
│     3 │ Person_3 │    33 │ City_B  │ 1707618109.6456246 │ NtYyz7yuXUWiEg │ Job_3      │
│     6 │ Person_6 │    36 │ City_B  │ 1707618109.6456246 │ z9LbuxL/2yaiiw │ Job_6      │
│     8 │ Person_8 │    38 │ City_B  │ 1707618109.6456246 │ eB5dHT6pOWnqxQ │ Job_8      │
│     4 │ 

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX